In [8]:
import pandas as pd
import numpy as np
import chardet
import io

def diagnostic_test(file_path):
    print(f"--- [1. 파일 로딩 시작: {file_path}] ---")
    
    # 인코딩 확인
    with open(file_path, 'rb') as f:
        raw = f.read()
        encoding = chardet.detect(raw)['encoding']
    print(f"인코딩 감지 결과: {encoding}")

    try:
        # 파일 읽기
        df = pd.read_csv(file_path, encoding=encoding, low_memory=False)
        # 모든 컬럼을 일단 문자열로 변환 (숫자형 헤더 방지)
        df.columns = [str(c) for c in df.columns]
        print(f"전체 데이터 모양: {df.shape}")
    except Exception as e:
        print(f"파일 로드 실패: {e}")
        return

    # LITE(RH) 기준 설정
    config = {"name": "Analog Mic", "type": "analog", "range": range(6, 47)}
    cols = df.columns[config["range"]]
    
    print(f"\n--- [2. 컬럼 및 규격 확인] ---")
    print(f"분석 범위 컬럼 샘플: {list(cols[:5])} ... {list(cols[-5:])}")
    
    # 1kHz 위치 찾기
    freqs = [float(str(c).split('.')[0]) for c in cols]
    idx_1k = np.argmin(np.abs(np.array(freqs) - 1000))
    col_1k = cols[idx_1k]
    print(f"1kHz로 판정된 컬럼명: '{col_1k}' (타입: {type(col_1k)})")
    
    limit_low_1k = df.iloc[0][col_1k]
    limit_high_1k = df.iloc[1][col_1k]
    print(f"1kHz 규격 값: Low({limit_low_1k}), High({limit_high_1k})")

    # 데이터 필터링
    sn_col = df.columns[3]
    test_data = df.iloc[2:].dropna(subset=[sn_col])
    test_data = test_data[test_data[sn_col].astype(str).str.contains('/', na=False)].reset_index(drop=True)
    print(f"유효 시료 수: {len(test_data)} EA")

    print(f"\n--- [3. 시료별 데이터 추출 테스트 (상위 5개)] ---")
    extracted_vals = []
    for idx, row in test_data.iterrows():
        raw_val = row[col_1k]
        num_val = pd.to_numeric(raw_val, errors='coerce')
        extracted_vals.append(num_val)
        
        if idx < 5:
            print(f"시료[{idx}] SN: {row[sn_col][:15]}... | 원본값: {raw_val} | 변환값: {num_val}")

    print(f"\n--- [4. 통계 계산 시뮬레이션] ---")
    v = np.array(extracted_vals)
    v_clean = v[~np.isnan(v)]
    
    print(f"추출된 전체 값 개수: {len(extracted_vals)}")
    print(f"NaN이 아닌 정상 숫자 개수: {len(v_clean)}")
    
    if len(v_clean) > 0:
        print(f"최종 결과 -> Min: {v_clean.min():.4f}, Max: {v_clean.max():.4f}, Avg: {v_clean.mean():.4f}")
    else:
        print("결과: 모든 값이 NaN입니다! 추출 과정에서 숫자로 변환되지 못했습니다.")
        
    print(f"\n--- [5. 데이터 타입 정밀 점검] ---")
    sample_raw = test_data[col_1k].iloc[0]
    print(f"데이터 원본 타입: {type(sample_raw)}")
    if isinstance(sample_raw, str):
        print(f"문자열 내 특수문자 포함 여부(ASCII): {[ord(c) for c in sample_raw]}")

# 실행
diagnostic_test('ok.csv')

--- [1. 파일 로딩 시작: ok.csv] ---
인코딩 감지 결과: GB2312
전체 데이터 모양: (275, 105)

--- [2. 컬럼 및 규격 확인] ---
분석 범위 컬럼 샘플: ['100.000', '112.000', '125.000', '140.000', '160.000'] ... ['6300.000', '7100.000', '8000.000', '9000.000', '10000.000']
1kHz로 판정된 컬럼명: '1000.000' (타입: <class 'str'>)
1kHz 규격 값: Low(-11.0), High(-9.0)
유효 시료 수: 260 EA

--- [3. 시료별 데이터 추출 테스트 (상위 5개)] ---
시료[0] SN: 96575GJ200/6A26... | 원본값: -10.419 | 변환값: -10.419
시료[1] SN: 96575GJ200/6A26... | 원본값: -10.397 | 변환값: -10.397
시료[2] SN: 96575GJ200/6A26... | 원본값: -10.365 | 변환값: -10.365
시료[3] SN: 96575GJ200/6A26... | 원본값: -10.512 | 변환값: -10.512
시료[4] SN: 96575GJ200/6A26... | 원본값: -10.308 | 변환값: -10.308

--- [4. 통계 계산 시뮬레이션] ---
추출된 전체 값 개수: 260
NaN이 아닌 정상 숫자 개수: 260
최종 결과 -> Min: -10.9190, Max: -9.7870, Avg: -10.3680

--- [5. 데이터 타입 정밀 점검] ---
데이터 원본 타입: <class 'numpy.float64'>
